## Simple REST Scoring Service for ONNX Models

__ Despite the presence of the Databricks logo :) , the whole point here is that we are not touching or using Spark in any way for this service.__

I'm just using this notebook environment for convenience, so you don't have to set up some other system today.

First we'll install the ONNXRuntime library

In [3]:
dbutils.library.installPyPI("onnxruntime")

Now we'll see how to score a record with it

In [5]:
import onnxruntime as rt
import numpy as np

session = rt.InferenceSession("/dbfs/FileStore/diamonds.onnx")

myDiamondToScore = 1.0 # carat weight

model_input = np.asarray([myDiamondToScore], dtype=np.float32).reshape(1,1)

session.run(["prediction"], {'carat':model_input}) #["prediction"] is the list of outputs ... you can pass None to get all

How did I know about the input shapes, types, and names? They're in the string repr of the graph (previous notebook) or...

In [7]:
for i in session.get_inputs():
  print(i.name, i.type, i.shape)
  
for i in session.get_outputs():
  print(i.name, i.type, i.shape)

Well, now the hard part is over ... let's pop this into a quick (and very dirty) service with `flask`

In [9]:
dbutils.library.installPyPI('flask')

In [10]:
from flask import Flask

app = Flask(__name__)

@app.route('/diamond/<float:carats>')
def score(carats):
  input0 = np.asarray([carats], dtype=np.float32).reshape(1,1)
  return str(session.run(['prediction'], {'carat':input0}))

In [11]:
from multiprocessing import Process

server = Process(target=app.run)
server.start()

In [12]:
%sh curl -s http://localhost:5000/diamond/1.0

In [13]:
%sh curl -s http://localhost:5000/diamond/1.5

In [14]:
server.terminate()